# Tables.jl integration example

This example demonstrates the Tables.jl integration in Fastback by running
a simple backtest and then converting account data to DataFrames for display.

The Tables.jl interface provides zero-copy views of trades, positions,
cash balances, equity balances, and collector output, making it easy to
export results to DataFrames.jl, CSV.jl, Arrow.jl or any other package
that consumes Tables-compatible sources.

In [1]:
using Fastback
using DataFrames
using Dates
using Random

# set RNG seed for reproducibility
Random.seed!(123);

# generate synthetic price series
N = 500;
prices = 100.0 .+ cumsum(randn(N) .* 0.5 .+ 0.05);
dts = map(x -> DateTime(2021, 1, 1) + Hour(x), 0:N-1);

# create trading account with $5'000 start capital (margin-enabled for shorting)
acc = Account(;
    funding=AccountFunding.Margined,
    base_currency=CashSpec(:USD),
    broker=FlatFeeBroker(; pct=0.001),
);
usd = cash_asset(acc, :USD)
deposit!(acc, :USD, 5_000.0);

# register instruments
AAPL = register_instrument!(acc, spot_instrument(Symbol("AAPL/USD"), :AAPL, :USD));
MSFT = register_instrument!(acc, spot_instrument(Symbol("MSFT/USD"), :MSFT, :USD));

# data collectors
collect_equity, equity_data = periodic_collector(Float64, Hour(12));
collect_drawdown, drawdown_data = drawdown_collector(DrawdownMode.Percentage, Hour(12));

# simple momentum strategy
let prev_price = prices[1]
    for (i, (dt, price)) in enumerate(zip(dts, prices))
        # trade every 10 hours based on price momentum
        if i % 10 == 0 && i > 10
            momentum = (price - prev_price) / prev_price

            if momentum > 0.02  # buy signal
                quantity = 10.0
                order = Order(oid!(acc), AAPL, dt, price, quantity)
                fill_order!(acc, order; dt=dt, fill_price=price, bid=price, ask=price, last=price)

            elseif momentum < -0.02  # sell signal
                quantity = -8.0
                order = Order(oid!(acc), MSFT, dt, price, quantity)
                fill_order!(acc, order; dt=dt, fill_price=price, bid=price, ask=price, last=price)
            end

            prev_price = price
        end

        # update positions P&L
        update_marks!(acc, AAPL, dt, price, price, price)
        update_marks!(acc, MSFT, dt, price, price, price)

        # collect equity data
        if should_collect(equity_data, dt)
            equity_value = equity(acc, usd)
            collect_equity(dt, equity_value)
            collect_drawdown(dt, equity_value)
        end
    end
end

# print account summary
show(acc)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ ACCOUNT SUMMARY ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Cash balances (1)
┌─────┬─────────┐
│     │   Value │
├─────┼─────────┤
│ USD │ 3480.51 │
└─────┴─────────┘

Equity balances (1)
┌─────┬─────────┐
│     │   Value │
├─────┼─────────┤
│ USD │ 5028.50 │
└─────┴─────────┘

Positions (2)
┌──────────┬────────┬──────────┬─────┬────────┐
│   Symbol │    Qty │ Entry px │ Ccy │    P&L │
├──────────┼────────┼──────────┼─────┼────────┤
│ AAPL/USD │  30.00 │   106.44 │ USD │ 123.99 │
│ MSFT/USD │ -16.00 │   104.91 │ USD │ -90.62 │
└──────────┴────────┴──────────┴─────┴────────┘

Trades (5)
┌────┬──────────┬─────────────────────┬───────┬────────┬────────┬────┬────┬─────┬──────────┬──────────┬────────────────┬──────────────┬───────┐
│ ID │   Symbol │                Date │   Qty │ Filled │  Price │ TP │ SL │ Ccy │ Fill P&L │   Cash Δ │ Return (gross) │ Return (net) │ Comm. │
├────┼──────────┼─

### Convert trades to DataFrame

In [2]:
df_trades = DataFrame(trades_table(acc))

println(df_trades)

5×19 DataFrame
 Row │ tid    oid    trade_date           order_date           symbol    settlement_style   side  fill_price  fill_qty  remaining_qty  take_profit  stop_loss  fill_pnl_settle  realized_qty  position_qty  position_price  commission_settle  cash_delta_settle  reason
     │ Int64  Int64  DateTime             DateTime             Symbol    T                  T     Float64     Float64   Float64        Float64      Float64    Float64          Float64       Float64       Float64         Float64            Float64            T
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │     1      1  2021-01-07T15:00:00  2021-01-07T15:00:00  AAPL/USD  PrincipalExchange  Buy      105.867      10.0            0.0          NaN        NaN              0.0           0.0      

### Convert positions to DataFrame

In [3]:
df_positions = DataFrame(positions_table(acc))

println(df_positions)

2×10 DataFrame
 Row │ index  symbol    qty      avg_entry_price  avg_settle_price  pnl_quote  base_ccy  quote_ccy  last_oid  last_tid
     │ Int64  Symbol    Float64  Float64          Float64           Float64    Symbol    Symbol     Int64     Int64
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │     1  AAPL/USD     30.0          106.438           106.438   123.994   AAPL      USD               4         4
   2 │     2  MSFT/USD    -16.0          104.907           104.907   -90.6229  MSFT      USD               5         5


### Convert cash balances to DataFrame

In [4]:
df_balances = DataFrame(balances_table(acc))

println(df_balances)

1×4 DataFrame
 Row │ index  symbol  balance  digits
     │ Int64  Symbol  Float64  Int64
─────┼────────────────────────────────
   1 │     1  USD     3480.51       2


### Convert equity balances to DataFrame

In [5]:
df_equities = DataFrame(equities_table(acc))

println(df_equities)

1×4 DataFrame
 Row │ index  symbol  equity   digits
     │ Int64  Symbol  Float64  Int64
─────┼────────────────────────────────
   1 │     1  USD      5028.5       2


### Convert equity collector data to DataFrame

In [6]:
df_equity_history = DataFrame(equity_data)

println(df_equity_history)

42×2 DataFrame
 Row │ date                 value
     │ DateTime             Float64
─────┼──────────────────────────────
   1 │ 2021-01-01T00:00:00  5000.0
   2 │ 2021-01-01T12:00:00  5000.0
   3 │ 2021-01-02T00:00:00  5000.0
   4 │ 2021-01-02T12:00:00  5000.0
   5 │ 2021-01-03T00:00:00  5000.0
   6 │ 2021-01-03T12:00:00  5000.0
   7 │ 2021-01-04T00:00:00  5000.0
   8 │ 2021-01-04T12:00:00  5000.0
   9 │ 2021-01-05T00:00:00  5000.0
  10 │ 2021-01-05T12:00:00  5000.0
  11 │ 2021-01-06T00:00:00  5000.0
  12 │ 2021-01-06T12:00:00  5000.0
  13 │ 2021-01-07T00:00:00  5000.0
  14 │ 2021-01-07T12:00:00  5000.0
  15 │ 2021-01-08T00:00:00  5012.25
  16 │ 2021-01-08T12:00:00  4982.47
  17 │ 2021-01-09T00:00:00  4979.38
  18 │ 2021-01-09T12:00:00  4941.13
  19 │ 2021-01-10T00:00:00  4958.25
  20 │ 2021-01-10T12:00:00  4958.7
  21 │ 2021-01-11T00:00:00  4963.23
  22 │ 2021-01-11T12:00:00  4973.7
  23 │ 2021-01-12T00:00:00  4941.93
  24 │ 2021-01-12T12:00:00  4946.5
  25 │ 2021-01-13T00:00:00  496

### Convert balance collector data to DataFrame

In [7]:
df_drawdown_history = DataFrame(drawdown_data)

println(df_drawdown_history)

42×2 DataFrame
 Row │ date                 drawdown
     │ DateTime             Float64
─────┼───────────────────────────────────
   1 │ 2021-01-01T00:00:00   0.0
   2 │ 2021-01-01T12:00:00   0.0
   3 │ 2021-01-02T00:00:00   0.0
   4 │ 2021-01-02T12:00:00   0.0
   5 │ 2021-01-03T00:00:00   0.0
   6 │ 2021-01-03T12:00:00   0.0
   7 │ 2021-01-04T00:00:00   0.0
   8 │ 2021-01-04T12:00:00   0.0
   9 │ 2021-01-05T00:00:00   0.0
  10 │ 2021-01-05T12:00:00   0.0
  11 │ 2021-01-06T00:00:00   0.0
  12 │ 2021-01-06T12:00:00   0.0
  13 │ 2021-01-07T00:00:00   0.0
  14 │ 2021-01-07T12:00:00   0.0
  15 │ 2021-01-08T00:00:00   0.0
  16 │ 2021-01-08T12:00:00  -0.00594133
  17 │ 2021-01-09T00:00:00  -0.00655759
  18 │ 2021-01-09T12:00:00  -0.0141898
  19 │ 2021-01-10T00:00:00  -0.0107733
  20 │ 2021-01-10T12:00:00  -0.0106828
  21 │ 2021-01-11T00:00:00  -0.00977928
  22 │ 2021-01-11T12:00:00  -0.00769129
  23 │ 2021-01-12T00:00:00  -0.0140289
  24 │ 2021-01-12T12:00:00  -0.0131167
  25 │ 2021-01-13T00